In [41]:
import pandas as pd

# Đọc file CSV gốc
df = pd.read_csv("train_data.csv")
df["Related_Knowledge"] = df["problem"].apply(lambda p: retrieve_context(p, k=3))
df.to_csv("train_with_knowledge.csv", index=False)



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [42]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/ziye2chen/LLMs-for-Mathematical-Analysis.git

In [ ]:
!pip install --upgrade pip
!pip install faiss-cpu sentence-transformers
!pip install unsloth transformers trl datasets pandas matplotlib rich scipy sentencepiece

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import torch
print("Pytorch version：")
print(torch.__version__)
print("CUDA Version: ")
print(torch.version.cuda)
print("cuDNN version is :")
print(torch.backends.cudnn.version())

Pytorch version：
2.7.1+cu126
CUDA Version: 
12.6
cuDNN version is :
90501


In [ ]:
import pandas as pd
import json
import faiss
from sentence_transformers import SentenceTransformer

# Đọc file train dữ liệu để dùng cho truy hồi
df = pd.read_csv("/content/train_data.csv")

# Gộp problem + solution thành văn bản corpus và thêm prefix "passage: "
texts = ("passage: " + df['problem'] + "\n\nLời giải:\n" + df['solution']).tolist()

# Lưu corpus
with open("rag_texts.json", "w", encoding="utf-8") as f:
    json.dump(texts, f, ensure_ascii=False, indent=2)

# Tạo model embedding
embedder = SentenceTransformer('intfloat/multilingual-e5-base')
embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True)

# Tạo FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Lưu index
faiss.write_index(index, "rag_index.faiss")


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Classifier

In [ ]:
alpaca_prompt = """Là trợ lý toán học, bạn cần phân tích bài toán để tìm ra loại bài toán đó trong Phân tích Thực tế. Cung cấp Problem_Type có thể được sử dụng để giải bài toán này.

### Problem:
{}

### Problem_Type:
{}"""


In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    Problem_Type = examples["problem_type"]
    Problem = examples["problem"]

    texts = []
    for problem, problem_type in zip(Problem, Problem_Type):
        text = alpaca_prompt.format(problem, problem_type) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}


from datasets import load_dataset
dataset = load_dataset('csv',data_files = '/content/train_data.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 300, # 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained("MathAnalysis_Qwen_Classifier") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_Classifier")
model.save_pretrained("/content/drive/MyDrive/MathAnalysis_Qwen_Classifier") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/MathAnalysis_Qwen_Classifier")

In [ ]:
test_prompt = """Là trợ lý toán học, bạn cần phân tích bài toán để tìm ra loại bài toán đó trong Phân tích Thực tế. Cung cấp Problem_Type có thể được sử dụng để giải bài toán này.

### Problem:
{}
"""

In [ ]:
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_Classifier", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# # alpaca_prompt = You MUST copy from above!

# inputs = tokenizer(
# [
#     test_prompt.format(
#         "Cho ba số dương \( a, b, c \) thoả mãn \( b \ne c \), \( \sqrt{a} + \sqrt{b} \ne \sqrt{c} \), \( a + b = (\sqrt{a} + \sqrt{b} - \sqrt{c})^2 \). Chứng minh đẳng thức: \[ \frac{a + (\sqrt{a} - \sqrt{c})^2}{b + (\sqrt{b} - \sqrt{c})^2} = \frac{\sqrt{a} - \sqrt{c}}{\sqrt{b} - \sqrt{c}}. \]", # Problem
#     )
# ], return_tensors = "pt").to("cuda")


In [ ]:
import json
import faiss
from sentence_transformers import SentenceTransformer

texts = json.load(open("rag_texts.json", "r", encoding="utf-8"))
index = faiss.read_index("rag_index.faiss")
embedder = SentenceTransformer('intfloat/multilingual-e5-base')

def retrieve_context(query, k=3):
    query_embedding = embedder.encode(["query: " + query])
    D, I = index.search(query_embedding, k)
    return [texts[i] for i in I[0]]

# STEP 3: Prompt + Truy xuất + Sinh lời giải
raw_problem = "Cho ba số dương \( a, b, c \) thoả mãn \( b \ne c \), \( \sqrt{a} + \sqrt{b} \ne \sqrt{c} \), \( a + b = (\sqrt{a} + \sqrt{b} - \sqrt{c})^2 \). Chứng minh đẳng thức: \[ \frac{a + (\sqrt{a} - \sqrt{c})^2}{b + (\sqrt{b} - \sqrt{c})^2} = \frac{\sqrt{a} - \sqrt{c}}{\sqrt{b} - \sqrt{c}}. \]"
contexts = retrieve_context(raw_problem, k=3)
context_block = "\n\n".join(contexts)

final_prompt = context_block + "\n\n" + test_prompt.format(raw_problem)


In [ ]:
# # Tự tính max_new_tokens an toàn
# input_len = inputs.input_ids.shape[-1]
# max_total_length = max_seq_length  # ví dụ: 10240
# safe_max_new_tokens = max_total_length - input_len

# Tokenize & sinh
inputs = tokenizer([final_prompt], return_tensors="pt").to("cuda")
input_len = inputs.input_ids.shape[-1]
safe_max_new_tokens = max_seq_length - input_len

# In thử kiểm tra
print(f"Input tokens: {input_len}, Max new tokens: {safe_max_new_tokens}")

# Dùng TextStreamer để in ra từng dòng sinh token
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Sinh kết quả
_ = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    streamer=text_streamer,
    max_new_tokens=safe_max_new_tokens,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import numpy as np

test_prompt_rag = """Bạn là trợ lý toán học. Dưới đây là một bài toán học cần phân loại Problem_Type.
Bạn có thể sử dụng các ví dụ tương tự từ dữ liệu dưới đây để hỗ trợ.

### Các ví dụ tương tự:
{}

### Problem:
{}

### Problem_Type:
"""


# Load mô hình đã fine-tune
# model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "/content/drive/MyDrive/MathAnalysis_Qwen_Classifier",
#         max_seq_length = 4096,
#         dtype = torch.float16,
#         load_in_4bit = True,
# )
# FastLanguageModel.for_inference(model)
streamer = TextStreamer(tokenizer, skip_prompt=True)

from sentence_transformers import SentenceTransformer
import faiss

# Load lại FAISS index và embedding model
kb_df = pd.read_csv("train_data.csv")
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
faiss_index = faiss.read_index("rag_index.faiss")

df = pd.read_csv("test_data.csv")

outputs = []
for idx, row in df.iterrows():
    problem = row["problem"]

    # Embed problem
    problem_emb = embedding_model.encode([problem])
    top_k = 3
    distances, indices = faiss_index.search(np.array(problem_emb), top_k)

    # Trích xuất context
    retrieved_examples = []
    for i in indices[0]:
        example = f"- Problem: {kb_df.iloc[i]['problem']}\n  Problem_Type: {kb_df.iloc[i]['problem_type']}"
        retrieved_examples.append(example)
    rag_context = "\n".join(retrieved_examples)

    # Format prompt với context
    prompt = test_prompt_rag.format(rag_context, problem)

    # Tokenize và generate
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    input_len = inputs.input_ids.shape[-1]
    safe_max_new_tokens = 4096 - input_len

    generated = model.generate(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
        max_new_tokens = safe_max_new_tokens,
        pad_token_id = tokenizer.eos_token_id,
    )

    output_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    outputs.append({
        "Problem": problem,
        "Predicted_ProblemType": output_text.strip()
    })

# Xuất file
df_out = pd.DataFrame(outputs)
df_out.to_csv("test_results_with_rag.csv", index=False)
print("✅ Đã lưu kết quả vào test_results_with_rag.csv")


# Problem Solver

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "auto",
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.8.1: Fast Qwen2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/Qwen2.5-Math-7B-bnb-4bit can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 2.5, it can be magically be extended to 10240!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False,  # ← TẮT ở đây
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
alpaca_prompt = """Bạn là trợ lý toán học. Hãy giải bài toán dưới đây với lời giải chi tiết, từng bước và lập luận toán học chặt chẽ.

Nếu bài toán yêu cầu sử dụng phương pháp $\epsilon$-$\delta$ (ví dụ: chứng minh giới hạn, tính liên tục), hãy đảm bảo bạn áp dụng đúng phương pháp.

Sử dụng ký hiệu toán học chính xác. Bạn có thể tham khảo các ví dụ sau, **nhưng KHÔNG được lặp lại nội dung đó.**

### Problem:
{}

### Problem_Type:
{}

### Retrieved_Examples:
{}

### --- Bài toán cần giải là dưới đây ---

### Solution:
{}"""



EOS_TOKEN = tokenizer.eos_token  # Đảm bảo thêm EOS để model dừng sinh

def formatting_prompts_func(examples):
    Problem_Type       = examples["problem_type"]
    Problem            = examples["problem"]
    Retrieved_Examples  = examples["Related_Knowledge"]
    Solution           = examples["solution"]
    texts = []
    for problem_type, problem, related_examples, solution in zip(Problem_Type, Problem, Retrieved_Examples, Solution):
        text = alpaca_prompt.format(problem, problem_type, related_examples, solution) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }


from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/train_with_knowledge.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/325 [00:00<?, ? examples/s]

In [8]:
len(dataset)

325

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 300, # 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"]:   0%|          | 0/325 [00:00<?, ? examples/s]

In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.705 GB of memory reserved.


In [11]:

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 325 | Num Epochs = 2 | Total steps = 300
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.010600
2,1.188800
3,0.879600
4,1.047200
5,1.119200
6,1.288200
7,0.890600
8,0.891600
9,0.835600
10,0.707800


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
model.save_pretrained("MathAnalysis_Qwen_ProblemSolver") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_ProblemSolver")
model.save_pretrained("/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver/tokenizer_config.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver/special_tokens_map.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver/vocab.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver/merges.txt',
 '/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver/added_tokens.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver/tokenizer.json')

In [1]:
import faiss
import json
from sentence_transformers import SentenceTransformer

# Load corpus và FAISS index
with open("rag_texts.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

index = faiss.read_index("rag_index.faiss")
embedder = SentenceTransformer('intfloat/multilingual-e5-base')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
def retrieve_context(problem, k=3):
    query = "query: " + problem
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k)
    retrieved = [texts[i] for i in I[0]]
    return "\n\n---\n\n".join([f"### Ví dụ {i+1}:\n{r}" for i, r in enumerate(retrieved)])


In [5]:
problem_prompt = """Bạn là trợ lý toán học. Hãy giải bài toán dưới đây với lời giải chi tiết, từng bước và lập luận toán học chặt chẽ.

Nếu bài toán yêu cầu sử dụng phương pháp $\epsilon$-$\delta$ (ví dụ: chứng minh giới hạn, tính liên tục), hãy đảm bảo bạn áp dụng đúng phương pháp.

Sử dụng ký hiệu toán học chính xác. Bạn có thể tham khảo các ví dụ sau, **nhưng KHÔNG được lặp lại nội dung đó.**

### Problem:
{}

### Problem_Type:
{}

### Retrieved_Examples:
{}

### --- Bài toán cần giải là dưới đây ---

### Solution:

"""

In [6]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/MathAnalysis_Qwen_ProblemSolver",
    max_seq_length = 8192,  # ví dụ
    dtype = "float16",
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)
model.eval()

# Nhập problem và problem_type
problem = "Tìm tất cả các giá trị thực của \( a \) và \( b \) sao cho: \[ (a^2 + 1)(b^2 + 1) = (a + 1)(b + 1)(ab + 1) \]"
problem_type = "Giải phương trình"

# Truy hồi ví dụ tương tự
retrieved_context = retrieve_context(problem, k=3)
# print("=== Retrieved Documents ===")
# print(retrieved_context)

# Tạo prompt
final_prompt = problem_prompt.format(problem, problem_type, retrieved_context)


==((====))==  Unsloth 2025.8.1: Fast Qwen2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/Qwen2.5-Math-7B-bnb-4bit can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 8192!


In [7]:
# Tính độ dài đầu vào an toàn
# input_len = inputs.input_ids.shape[-1]
# max_total_length = max_seq_length  # ví dụ: 10240
# safe_max_new_tokens = max_total_length - input_len

# Tokenize
inputs = tokenizer([final_prompt], return_tensors="pt").to("cuda")
input_len = inputs.input_ids.shape[-1]
max_new_tokens = 8192 - input_len  # Giới hạn token sinh thêm

print(f"Input tokens: {input_len}, Max new tokens: {max_new_tokens}")

# Stream kết quả
streamer = TextStreamer(tokenizer, skip_prompt=True)


_ = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    streamer=streamer,
    max_new_tokens=max_new_tokens,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)


Input tokens: 941, Max new tokens: 7251
Xét hiệu:
(a^{2} + 1)(b^{2} + 1) − 4ab
= a^{2}b^{2} + a^{2} + b^{2} + 1 − 4ab
= (a^{2}b^{2} − 2ab + 1) + (a^{2} − 2ab + b^{2})
= (ab − 1)^{2} + (a − b)^{2}.
Với hai số thực a, b tùy ý, ta có: (ab − 1)^{2} ≥ 0 và (a − b)^{2} ≥ 0.
Suy ra (a^{2} + 1)(b^{2} + 1) − 4ab ≥ 0.
#### Kết quả: (a^{2} + 1)(b^{2} + 1) ≥ 4ab.<|endoftext|>


In [10]:
import pandas as pd
import pickle

# Load file chứa các bài toán và lời giải
df = pd.read_csv("train_data.csv")  # hoặc bất kỳ file nào chứa các lời giải mẫu

# Lấy cột solution làm nguồn kiến thức
knowledge_texts = df["solution"].tolist()

# Ghi vào file knowledge_texts.pkl
with open("knowledge_texts.pkl", "wb") as f:
    pickle.dump(knowledge_texts, f)

print("✅ Đã tạo file knowledge_texts.pkl")


✅ Đã tạo file knowledge_texts.pkl


In [12]:
import pandas as pd
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch
from sentence_transformers import SentenceTransformer, util
import faiss
import pickle

# === Cấu hình ===
model_name = "MathAnalysis_Qwen_ProblemSolver"
max_seq_length = 4096
dtype = torch.float16
load_in_4bit = True

# === PROMPT ===
problem_prompt = """Với tư cách là trợ lý toán học, hãy giải bài toán sau. Đưa ra lời giải chi tiết, từng bước bằng lập luận toán học chặt chẽ. Nếu bài toán yêu cầu sử dụng phương pháp $\epsilon$-$\delta$, hãy áp dụng đúng. Sử dụng ký hiệu toán học chính xác.

### Problem:
{}

### Problem_Type:
{}

### Related_Knowledge:
{}

### Solution:
"""

# === Load FAISS index và dữ liệu knowledge base ===
with open("knowledge_texts.pkl", "rb") as f:
    knowledge_texts = pickle.load(f)  # List[str]

# Load embedding model
embed_model = SentenceTransformer("intfloat/multilingual-e5-base")

# Vector hóa toàn bộ kiến thức
embeddings = embed_model.encode(knowledge_texts, convert_to_numpy=True, show_progress_bar=True)

# Tạo FAISS index
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)

# Lưu index vào file
faiss.write_index(faiss_index, "faiss_index.bin")

faiss_index = faiss.read_index("faiss_index.bin")
embed_model = SentenceTransformer("intfloat/multilingual-e5-base")

# === Hàm truy xuất ngữ cảnh từ knowledge base ===
def retrieve_context(problem, problem_type, top_k=3):
    query = f"{problem} | {problem_type}"
    embedding = embed_model.encode(query, convert_to_tensor=True, normalize_embeddings=True).cpu().numpy()
    D, I = faiss_index.search(embedding.reshape(1, -1), top_k)
    context_passages = [knowledge_texts[i] for i in I[0] if i < len(knowledge_texts)]
    return "\n".join(context_passages)

# === Load model ===
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name=model_name,
#     max_seq_length=max_seq_length,
#     dtype=dtype,
#     load_in_4bit=load_in_4bit,
#     device_map="auto",
#     llm_int8_enable_fp32_cpu_offload=True
# )
# FastLanguageModel.for_inference(model)

# === Load file CSV ===
df = pd.read_csv("test_data.csv")
print(f"Tổng số câu: {len(df)}")

# === Chuẩn bị lưu kết quả ===
results = []

# === Duyệt từng câu ===
for idx, row in df.iterrows():
    problem = row['problem']
    problem_type = row['problem_type']

    context = retrieve_context(problem, problem_type)  # Truy xuất từ FAISS

    # Ghép thành prompt hoàn chỉnh
    prompt = problem_prompt.format(problem, problem_type, context)
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    input_len = inputs.input_ids.shape[-1]
    safe_max_new_tokens = max_seq_length - input_len

    print("=" * 50)
    print(f"▶️ Bài {idx+1}/{len(df)} | Input tokens: {input_len} | Max new: {safe_max_new_tokens}")

    # Stream
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)

    output_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        streamer=text_streamer,
        max_new_tokens=safe_max_new_tokens,
        pad_token_id=tokenizer.eos_token_id
    )

    # Lấy kết quả
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    solution = decoded[len(prompt):].strip()  # Bỏ phần prompt gốc

    results.append({
        "problem": problem,
        "problem_type": problem_type,
        "related_knowledge": context,
        "solution": solution
    })

# === Lưu ra CSV ===
results_df = pd.DataFrame(results)
results_df.to_csv("solutions_with_rag.csv", index=False, encoding="utf-8-sig")

print("\n✅ Đã lưu toàn bộ lời giải vào file solutions_with_rag.csv")


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Tổng số câu: 50
▶️ Bài 1/50 | Input tokens: 698 | Max new: 3398


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


a) Với m = 1: 
( 2x^{2} – 3x + 1 = 0 
Rightarrow x = 1 
) hoặc 
( x = 
frac{1}{2}. 
)

b) Để là phương trình bậc hai: 
( m + 1 
neq 0 
Rightarrow m 
neq -1. 
)

c) ∆ = 9 – 4(m + 1) = -4m + 5.
- Hai nghiệm phân biệt khi 
( m < 
frac{5}{4}. 
)
- Nghiệm kép khi 
( m = 
frac{5}{4}. 
)
- Vô nghiệm khi 
( m > 
frac{5}{4}. 
)
#### Kết quả: a) x = 1 hoặc x = 1/2; b) m ≠ -1; c) Điều kiện như trên.<|endoftext|>
▶️ Bài 2/50 | Input tokens: 1397 | Max new: 2699
Với \( b \ne c \), \( a + b = (\sqrt{a} + \sqrt{b} - \sqrt{c})^2 \), ta có:
\[ a + b = a + b + c + 2\sqrt{ab} - 2\sqrt{ac} - 2\sqrt{bc} \Rightarrow c + 2\sqrt{ab} = 2\sqrt{ac} + 2\sqrt{bc}. \]
Do đó:
\[ \frac{a + (\sqrt{a} - \sqrt{c})^2}{b + (\sqrt{b} - \sqrt{c})^2} = \frac{a + a + c - 2\sqrt{ac}}{b + b + c - 2\sqrt{bc}} = \frac{2a + c - 2\sqrt{ac}}{2b + c - 2\sqrt{bc}}. \]
Đặt \( \sqrt{a} = x, \sqrt{b} = y, \sqrt{c} = z \), ta có:
\[ \frac{2x^2 + z^2 - 2xz}{2y^2 + z^2 - 2yz} = \frac{x(x - z)}{y(y - z)}. \]
Do \( a + b = (\sqrt{a} + \sqrt{b

KeyboardInterrupt: 